# Lecture 3 背包问题系列

## 01背包问题

一共有 $N$ 件物品，第 $i$（ $i$ 从 $1$ 开始）件物品的重量为 $w[i]$，价值为 $v[i]$。在总重量不超过背包承载上限 $W$ 的情况下，能够装入背包的最大价值是多少？

状态空间: $(n, C), n = 1, 2, ... N, C = 0, ..., C$

In [23]:
def knakpack_01(w, v, W, ret_items=False):
    """
    Use dp to solve the 0-1 knapsack problem

    :param w: (list<int>) the weight of each item
    :param v: (list<int>) the value of each item
    :param W: (int) the capacity of the knapsack
    :return: (int) the maximum value of the knapsack
    """
    N = len(w)
    # dp[i][j] represents the maximum value of the knapsack with capacity j and start selecting from ith item, we finally want dp[0][W]
    dp = [[0]*(W+1) for _ in range(N)] # shape (N, W+1)

    # dp init, calc dp[N-1][*]
    for capacity in range(w[N-1], W+1):
        dp[N-1][capacity] = v[N-1]

    # dp sweep, from N-2 to 0
    for i in range(N-2, -1, -1):
        for capacity in range(W+1):
            dp[i][capacity] = dp[i+1][capacity]
            if w[i] <= capacity:
                dp[i][capacity] = max(dp[i][capacity], dp[i+1][capacity-w[i]]+v[i])

    if ret_items:
        items= knakpack_path(w, dp)
        return dp[0][W], items

    return dp[0][W]

def knakpack_path(w, dp):
    """
    Reconstruction the list of items from the dp table

    :param w:
    :param dp: (list<list<int>>) the dp table, shape: (N, W+1)
    :return: (list<int>) the path of the knapsack
    """
    N, C = len(dp), len(dp[0])-1
    items = []
    for i in range(N):
        if i == N-1 and dp[i][C] != 0:
            items.append(N-1)
            break
        if dp[i][C] != dp[i+1][C]:
            C -= w[i]
            items.append(i)
    return items

01背包-测试数据

In [24]:
# # toy dataset
# w = (3, 5, 2, 4)
# v = (5, 7, 3, 10)
# W = 8
# expected = 15

w = (95, 4, 60, 32, 23, 72, 80, 62, 65, 46)
v = (55, 10, 47, 5, 4, 50, 8, 61, 85, 87)
W = 269
expected = 295

best_value, items = knakpack_01(w,v, W, ret_items=True)
best_value, items

(295, [1, 2, 3, 7, 8, 9])

## 完全背包问题

一共有 $N$ 种物品，第 $i$（ $i$ 从 $1$ 开始）种物品的重量为 $w[i]$，价值为 $v[i]$, 每种物品可以有无限多个。在总重量不超过背包承载上限 $W$ 的情况下，能够装入背包的最大价值是多少？

**思路 1** 转化为 01 背包

我们可以考虑将第 $i$ 种物品复制 $k$ 份，$k = W / w[i]$。这样，我们就将完全背包问题转化为了 01 背包问题。但是空间复杂度较大，最坏的空间复杂度为 $O(N*W*W)$

当然，我们可以参考数制的思想， 比如第 $i$ 种物品可以最多装 6 个, 我们不需要创建 6 个相同的 item i, 相反, 我们可以创建 3 个 item i, 一个 item $i*2$, 一个 item $i*4$。这样，我们就可以将空间复杂度降低到 $O(N*W*log(W))$。

**思路 2** 修改状态转移方程

在 01 背包问题中， 状态转移方程为:
  $$dp[i][C] = max(dp[i+1][C], v[i] + dp[i+1][C-w[i]])$$
前一项代表不选择该物品，后一项代表选择该物品。后一项中的 $i+1$ 表示我们选择了该商品，接下来就只能从后面的物品中选择了。而在完全背包问题中，我们只需要去掉这一限制即可。

因此，在完全背包问题中， 状态转移方程为:
  $$dp[i][C] = max(dp[i+1][C], v[i] + dp[i][C-w[i]])$$

In [45]:
def knakpack_complete(w, v, W):
    """
    Use dp to solve the complete knapsack problem

    :param w: (list<int>) the weight of each item
    :param v: (list<int>) the value of each item
    :param W: (int) the capacity of the knapsack
    :return: (int) the maximum value of the knapsack
    """
    N = len(w)
    # dp[i][j] represents the maximum value of the knapsack with capacity j and start selecting from ith item, we finally want dp[0][W]
    dp = [[0]*(W+1) for _ in range(N)] # shape (N, W+1)

    # dp init, calc dp[N-1][*]
    for capacity in range(w[N-1], W+1):
        n_items = capacity // w[N-1]
        dp[N-1][capacity] = v[N-1] * n_items

    # dp sweep, from N-2 to 0
    for i in range(N-2, -1, -1):
        for capacity in range(W+1):
            dp[i][capacity] = dp[i+1][capacity]
            if w[i] <= capacity:
                dp[i][capacity] = max(dp[i][capacity], dp[i][capacity-w[i]]+v[i])
    return dp[0][W]

测试完全背包问题

In [47]:
# toy dataset
w = (3, 5, 2, 3)
v = (5, 7, 6, 7)
W = 9
expected = 25

knakpack_complete(w, v, W, ret_items=False)

25

## 多重背包问题

多重背包（bounded knapsack problem）与前面不同就是每种物品是有限个: 一共有 $N$ 种物品, 第 $i$（$i$ 从 $1$ 开始）种物品的数量为 $n[i]$ ，重量为 $w[i]$, 价值为$v[i]$ .在总重量不超过背包承载上限 $W$ 的情况下，能够装入背包的最大价值是多少？

**思路 1** 转化为 01 背包

类似前面的解法, 我们可以把第 $i$ 种物品复制 $k$ 份，$k = min(n[i], W / w[i])$。这样，我们就将多重背包问题转化为了 01 背包问题。但是空间复杂度较大，最坏的空间复杂度为 $O(N*W*W)$。

借鉴数制的思想, 我们可以仅使用 $log(min(n[i], W / w[i]))$ 个物品 $i$, 从而将空间复杂度降低到 $O(N*W*log(W))$。

**思路 2** 修改状态转移方程

对于第 i 个物品, 我们的动作空间在 0-1 背包中只有 2 个动作, 而现在有 $min(n[i], W / w[i])$ 个动作。因此，我们可以将状态转移方程修改为:
  $$dp[i][C] = max{(dp[i+1][C], v[i]*k + dp[i+1][C-w[i]*k]), k=1,...}$$

In [68]:
def knakpack_multi(w, v, n, W):
    """
    Use dp to solve the multi knapsack problem

    :param w: (list<int>) the weight of each item
    :param v: (list<int>) the value of each item
    :param n: (list<int>) the number of each item
    :param W: (int) the capacity of the knapsack
    :return: (int) the maximum value of the knapsack
    """
    N = len(w)
    # dp[i][j] represents the maximum value of the knapsack with capacity j and start selecting from ith item, we finally want dp[0][W]
    dp = [[0]*(W+1) for _ in range(N)] # shape (N, W+1)

    # dp init, calc dp[N-1][*]
    for capacity in range(w[N-1], W+1):
        n_avai = capacity // w[N-1]
        dp[N-1][capacity] = v[N-1] * min(n_avai, n[N-1])

    # dp sweep, from N-2 to 0
    for i in range(N-2, -1, -1):
        for capacity in range(W+1):
            dp[i][capacity] = dp[i+1][capacity]
            for k in range(1, min(n[i]+1, capacity//w[i]+1)):
                dp[i][capacity] = max(dp[i][capacity], dp[i+1][capacity-w[i]*k]+v[i]*k)
    return dp[0][W]

In [70]:
# toy dataset
w = (3, 5, 2, 3)
v = (5, 7, 6, 7)
n = (1, 1, 1, 2)
W = 9

expected = 20
knakpack_multi(w, v, n, W)

20